<a href="https://colab.research.google.com/github/MamiyaA/Biomechanics-and-function-of-proprioceptors-Python-analysis-functions/blob/master/Filter_ScanImage_tiff_files_average_fastZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Read in .tiff files acquired using the ScanImage (3D fast Z setting) and filter them using a gaussian 3D filter

- Find all the .tif files (acquired by the ScanImage)
- Separate the images into the GCaMP signal and tdTomato signal
- Separate the GCaMP and tdTomato signals into each depth
- Apply 3D gaussian filter (x, y, and time) to each signal at each depth
- Average the signal over the depth at each time point (for this genotype we have 4 steps, 3 micron/step)
- Save the average image (pickel file)

### Run functions in google colab using data stored on google drive


### Connect to a hosted runtime

### Install ScanImage Tiff Reader provided by Vidriotech to read in the ScanImage files

In [ ]:
#Install ScanImage Tiff Reader for Python
!pip install scanimage-tiff-reader

In [ ]:
#import necessary packages.
from ScanImageTiffReader import ScanImageTiffReader
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import pickle

### Mount Google Drive (assuming we have data on google drive)

In [ ]:
#link to the google drive. Need to follow the authorization instructions.
from google.colab import drive
drive.mount('/content/drive')

### Get the path to the imaging data (.tiff files)

In [ ]:
#Get the file path to the imaging data

image_file_path=[]
#Path for the image files
path = '/content/drive/Shared drives/DataDirectory'

#Go through all the files in a directory
for file_name in os.listdir(path):
    if fnmatch.fnmatch(file_name,'2*.tif'):
        image_file_path.append(os.path.join(path,file_name))
        
image_file_path_sorted=sorted(image_file_path)

#Print to make sure
for f in image_file_path_sorted:
    print(f)
    


### Define a function for reading the ScanImage data (Fast-Z stack) and apply 3D gaussian filter 

In [ ]:
#Function for:
#1) Read in the .tiff file
#2) Separate into GCaMP and tdTomato signals
#3) Separate into each depth
#4) Apply 3D gaussian filter (x, y, time)
#5) Average over the depth for each time point.
#6) Save as a pickel file
# NofZ specify the Fast-z levels in the image.
# FileName is: ImageFilesPathSorted[0], etc.
# Recommended Gaussian_sigma_array = [5, 5, 5]

def apply_3D_gaussian_to_ScanImageFile(FileName,NofZ,gaussian_sigma_array):
    
    #Load the image using ScanImageTiffReader
    TimeSeries=ScanImageTiffReader(FileName).data()
    #Close the file
    ScanImageTiffReader(FileName).close()
    
    #Currently the images are multiplexed so NofFrames*NoChannels*NofZlevels
    #is the first dimension.

    #We first split into two channels because we know they all have two channels
    #for the 1st channel (start with 1 and take every other frame)
    Channel_1_Index=np.arange(0, TimeSeries.shape[0],2)
    Channel_2_Index=np.arange(1,TimeSeries.shape[0],2)

    #assuming GCaMP is channel 1 and tdT is channel 2
    #This is true for all downstairs experiments

    GCaMPSignal=TimeSeries[Channel_1_Index]
    TdTomatoSignal=TimeSeries[Channel_2_Index]

    #Split into different z-levels and apply 3D gaussian filter
    #First for the GCaMP signal.
    depth_avg_image_GCaMP=np.zeros((GCaMPSignal.shape[0]//NofZ,GCaMPSignal.shape[1],GCaMPSignal.shape[2]),dtype=np.int16)

    for depth in range(NofZ):
        depthIndex=np.arange(depth,GCaMPSignal.shape[0],NofZ)
        temp_filtered = gaussian_filter(GCaMPSignal[depthIndex], sigma=gaussian_sigma_array)
    
        depth_avg_image_GCaMP=np.add(depth_avg_image_GCaMP,temp_filtered)
        del temp_filtered
    
    #save the depth_avg_image
    depth_avg_image_GCaMP=depth_avg_image_GCaMP/NofZ
    image_file_name=FileName.split('.')
    outfile_name=(image_file_name[0]+"GCaMP_Filtered")

    outfile=open(outfile_name,'wb')
    pickle.dump(depth_avg_image_GCaMP,outfile)
    outfile.close()
    del depth_avg_image_GCaMP

    #Do the same for the tdTomato signal.
    depth_avg_image_tdTomato=np.zeros((TdTomatoSignal.shape[0]//NofZ,TdTomatoSignal.shape[1],TdTomatoSignal.shape[2]),dtype=np.int16)

    for depth in range(NofZ):
        depthIndex=np.arange(depth,TdTomatoSignal.shape[0],NofZ)
        temp_filtered = gaussian_filter(TdTomatoSignal[depthIndex], sigma=gaussian_sigma_array)
    
        depth_avg_image_tdTomato=np.add(depth_avg_image_tdTomato,temp_filtered)
        del temp_filtered
    
    #save the depth_avg_image
    depth_avg_image_tdTomato=depth_avg_image_tdTomato/NofZ
    image_file_name=FileName.split('.')
    outfile_name=(image_file_name[0]+"tdTomato_Filtered")

    outfile=open(outfile_name,'wb')
    pickle.dump(depth_avg_image_tdTomato,outfile)
    outfile.close()
    del depth_avg_image_tdTomato


### Run for all the files in the Data directory

In [ ]:
#Go throuh all the files in the data directory
for inputFiles in image_file_path_sorted:
    print(inputFiles)
    apply_3D_gaussian_to_ScanImageFile(inputFiles,4,[5,5,5])

    